In [1]:
import pandas as pd
import numpy as np
from prog.tool import *
from prog.model import *
import warnings, pickle, os, json
warnings.filterwarnings("ignore")

aluminum_division = 12
random_state = 99
weight_limit = 56
l_extra = 6.4
f_extra = 8

In [2]:
input_ = {
    "work_id": "julietest",
    "op": 1,
    "model_id": "20230807120000000",
    "speed": "590",
    "l_angle_ori": "23",
    "l_weight_ori": "130",
    "f_angle_ori": "75",
    "f_weight_ori": "155",
    "material": "aluminum",
    "adjust": "2"
}

In [3]:
def adjust_angle(angle, adjustment):
    angle += adjustment
    if angle < 0:
        angle += 360

    elif angle >= 360:
        angle -= 360

    return round(angle)

In [4]:
adjustments = {
    1: -15, 
    2: -7.5,
    3: 0, 
    4: 7.5, 
    5: 15
}

In [5]:
work_id      = input_["work_id"]
op           = input_["op"]
model_id     = input_["model_id"]
speed        = int(input_["speed"])
adjust       = int(input_["adjust"])
l_angle_ori = int(input_["l_angle_ori"])
l_angle_ori1  = adjust_angle(l_angle_ori, adjustments[adjust])
l_weight_ori = float(input_["l_weight_ori"])
f_angle_ori = int(input_["f_angle_ori"])
f_angle_ori1  = adjust_angle(f_angle_ori, adjustments[adjust])
f_weight_ori = float(input_["f_weight_ori"])
material     = input_["material"]

In [6]:
root = r"C:\Users\tzuli\Documents\python\teco"

# 取得predict位置
pred_path = os.path.join(root, "data", "predict")        
os.makedirs(pred_path, exist_ok = True)
output_json = os.path.join(pred_path, "output.json")

# 取得model位置     
model_detail = os.path.join(root, "data", "train", model_id, "model")

config_path = os.path.join(root, "prog", "config.json")
with open(config_path) as f:
    config = json.load(f)

### Load model

In [7]:
side = "L"
model_l = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

side = "F"
model_f = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

In [8]:
df_l = pd.DataFrame([[l_angle_ori1, l_weight_ori]], columns = ["初始_L側角度", "初始_L側不平衡量"])
df_f = pd.DataFrame([[f_angle_ori1, f_weight_ori]], columns = ["初始_F側角度", "初始_F側不平衡量"])

In [9]:
df_l, df_f

(   初始_L側角度  初始_L側不平衡量
 0       16      130.0,
    初始_F側角度  初始_F側不平衡量
 0       68      155.0)

In [10]:
# 超過60補上特定值
if df_l.loc[0, "初始_L側不平衡量"] > weight_limit:
    df_l.loc[0, "初始_L側不平衡量"] += l_extra

if df_f.loc[0, "初始_F側不平衡量"] > weight_limit:
    df_f.loc[0, "初始_F側不平衡量"] += f_extra

In [11]:
df_l, df_f

(   初始_L側角度  初始_L側不平衡量
 0       16      136.4,
    初始_F側角度  初始_F側不平衡量
 0       68      163.0)

In [12]:
angle_init = np.linspace(0, 360, (aluminum_division + 1)).astype(int)
print(f"angle_init = {angle_init}")

df_l = calculate_angle_proportion(df_l, angle_init, aluminum_division, side = "L")            
df_f = calculate_angle_proportion(df_f, angle_init, aluminum_division, side = "F")

angle_init = [  0  30  60  90 120 150 180 210 240 270 300 330 360]


In [13]:
df_l = calculate_weight(df_l, side = "L")
df_f = calculate_weight(df_f, side = "F")

In [14]:
df_l.head()

,初始_L側角度,初始_L側不平衡量,angle_proportion,angle_0,angle_target,prop_37,3_raw,7_raw,5_raw,10_raw,...,7_amt,5_amt,10_amt,3_solution,7_solution,5_solution,10_solution,37_delta,5_delta,10_delta
0,16,136.4,0.466667,0.0,55,37,40.92,95.48,68.2,136.4,...,1,1,2,"{'5': '1', '16': '1', '20': '1'}","{'7': '1', '16': '2'}","{'5': '1', '7': '1'}",{'20': '2'},-0.4,-0.2,-0.4


In [15]:
df_f.head()

,初始_F側角度,初始_F側不平衡量,angle_proportion,angle_0,angle_target,prop_37,3_raw,7_raw,5_raw,10_raw,...,7_amt,5_amt,10_amt,3_solution,7_solution,5_solution,10_solution,37_delta,5_delta,10_delta
0,68,163.0,0.266667,0.0,37,73,48.9,114.1,81.5,163.0,...,1,1,2,"{'4': '1', '5': '1', '20': '2'}",{},"{'6': '1', '20': '1'}","{'7': '2', '20': '2'}",-2.0,0.5,1.0


In [16]:
df_l1 = encoding(df_l, side = "L")
df_f1 = encoding(df_f, side = "F")

In [17]:
X_l, y_l = split_data(df_l1, random_state, predict = True)
X_f, y_f = split_data(df_f1, random_state, predict = True)

In [18]:
X_l.head()

,angle_proportion,angle_0,3,7,37_delta,5,5_delta,10,10_delta
0,0.466667,0.0,0.300587,0.285924,-0.4,0.087977,-0.2,0.293255,-0.4


In [19]:
X_f.head()

,angle_proportion,angle_0,3,7,37_delta,5,5_delta,10,10_delta
0,0.266667,0.0,0.300613,0.0,-2.0,0.159509,0.5,0.331288,1.0


In [20]:
y_pred_l = model_l.predict(X_l)[0]
y_pred_f = model_f.predict(X_f)[0]
print(f"real - l: {y_l.values[0]}, f: {y_f.values[0]}")
print(f"pred - l: {y_pred_l}, f: {y_pred_f}")

real - l: 1, f: 0
pred - l: 1, f: 0


In [27]:
l_ans

初始_L側角度                                           16
初始_L側不平衡量                                      136.4
angle_proportion                            0.466667
angle_0                                          0.0
angle_target                                      55
prop_37                                           37
3_raw                                          40.92
7_raw                                          95.48
5_raw                                           68.2
10_raw                                         136.4
3_full                                             0
7_full                                             1
5_full                                             1
10_full                                            1
3                                                 41
7                                                 39
5                                                 12
10                                                40
3_amt                                         

In [28]:
56+80

136

In [22]:
l_ans = df_l.iloc[0]
f_ans = df_f.iloc[0]

proportion = {    
    "1:0": ["10"],
    "5:5": ["5", "5"],
    "3:7": ["3", "7"]
}
predicts = {}
for key, prop in proportion.items():
    l_weight_pred = [str(round((l_ans[f"{prop[0]}_full"] * weight_limit) + (l_ans[prop[0]] * l_ans[f"{prop[0]}_amt"])))]
    f_weight_pred = [str(round((f_ans[f"{prop[0]}_full"] * weight_limit) + (f_ans[prop[0]] * f_ans[f"{prop[0]}_amt"])))]

    if prop != ["10"]:
        l_weight_pred += [str(round((l_ans[f"{prop[1]}_full"] * weight_limit) + (l_ans[prop[1]] * l_ans[f"{prop[1]}_amt"])))]
        f_weight_pred += [str(round((f_ans[f"{prop[1]}_full"] * weight_limit) + (f_ans[prop[1]] * f_ans[f"{prop[1]}_amt"])))]

    predict = {
        "l_angle_pred":  str(l_angle_ori),
        "l_weight_pred": l_weight_pred,
        "l_comb": {},
        "f_angle_pred":  str(f_angle_ori),
        "f_weight_pred": f_weight_pred,
        "f_comb": {},
    }

    fill_solution(prop, l_ans, predict, l_weight_pred, weight_limit, side = "l")
    fill_solution(prop, f_ans, predict, f_weight_pred, weight_limit, side = "f")

    predicts[key] = predict
    
predicts

{'1:0': {'l_angle_pred': '23',
  'l_weight_pred': ['136'],
  'l_comb': {'136': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '40': {'count': '2', 'comb': {'20': '2'}}}},
  'f_angle_pred': '75',
  'f_weight_pred': ['164'],
  'f_comb': {'164': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '54': {'count': '2', 'comb': {'7': '2', '20': '2'}}}}},
 '5:5': {'l_angle_pred': '23',
  'l_weight_pred': ['68', '68'],
  'l_comb': {'68': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '12': {'count': '1', 'comb': {'5': '1', '7': '1'}}}},
  'f_angle_pred': '75',
  'f_weight_pred': ['82', '82'],
  'f_comb': {'82': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '26': {'count': '1', 'comb': {'6': '1', '20': '1'}}}}},
 '3:7': {'l_angle_pred': '23',
  'l_weight_pred': ['41', '95'],
  'l_comb': {'41': {'41': {'count': '1',
     'comb': {'5': '1', '16': '1', '20': '1'}}},
   '95': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '39': {'count': '1', 

In [23]:
# 確認37分的方向
if l_ans["prop_37"] == "73":
    predicts["3:7"]["l_weight_pred"] = predicts["3:7"]["l_weight_pred"][::-1]
elif f_ans["prop_37"] == "73":
    predicts["3:7"]["f_weight_pred"] = predicts["3:7"]["f_weight_pred"][::-1]
predicts

{'1:0': {'l_angle_pred': '23',
  'l_weight_pred': ['136'],
  'l_comb': {'136': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '40': {'count': '2', 'comb': {'20': '2'}}}},
  'f_angle_pred': '75',
  'f_weight_pred': ['164'],
  'f_comb': {'164': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '54': {'count': '2', 'comb': {'7': '2', '20': '2'}}}}},
 '5:5': {'l_angle_pred': '23',
  'l_weight_pred': ['68', '68'],
  'l_comb': {'68': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '12': {'count': '1', 'comb': {'5': '1', '7': '1'}}}},
  'f_angle_pred': '75',
  'f_weight_pred': ['82', '82'],
  'f_comb': {'82': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '26': {'count': '1', 'comb': {'6': '1', '20': '1'}}}}},
 '3:7': {'l_angle_pred': '23',
  'l_weight_pred': ['41', '95'],
  'l_comb': {'41': {'41': {'count': '1',
     'comb': {'5': '1', '16': '1', '20': '1'}}},
   '95': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
    '39': {'count': '1', 

In [24]:
y_map = {0: "3:7", 1: "5:5", 2: "1:0"}
best = {
    "l_side": y_map[y_pred_l],
    "f_side": y_map[y_pred_f]
}
best

{'l_side': '5:5', 'f_side': '3:7'}

In [25]:
result = {
    "status":   "success",
    "reason":   "",
    "work_id":  work_id,
    "op":       op,
    "model_id": model_id,
    "predict":  predicts,
    "best": best
}
result

{'status': 'success',
 'reason': '',
 'work_id': 'julietest',
 'op': 1,
 'model_id': '20230807120000000',
 'predict': {'1:0': {'l_angle_pred': '23',
   'l_weight_pred': ['136'],
   'l_comb': {'136': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
     '40': {'count': '2', 'comb': {'20': '2'}}}},
   'f_angle_pred': '75',
   'f_weight_pred': ['164'],
   'f_comb': {'164': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
     '54': {'count': '2', 'comb': {'7': '2', '20': '2'}}}}},
  '5:5': {'l_angle_pred': '23',
   'l_weight_pred': ['68', '68'],
   'l_comb': {'68': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
     '12': {'count': '1', 'comb': {'5': '1', '7': '1'}}}},
   'f_angle_pred': '75',
   'f_weight_pred': ['82', '82'],
   'f_comb': {'82': {'56': {'count': '1', 'comb': {'16': '1', '20': '2'}},
     '26': {'count': '1', 'comb': {'6': '1', '20': '1'}}}}},
  '3:7': {'l_angle_pred': '23',
   'l_weight_pred': ['41', '95'],
   'l_comb': {'41': {'41': {'count': '1',
  

In [26]:
with open(output_json, 'w') as file:
    json.dump(result, file, indent = 4)